In [13]:
import pandas as pd

In [14]:
# The data files is stored in the data directory under the root directory of this repo. 
data_dir = "../../data/data/dataanalysis/"
# Name of the file being read
case_sheet = "bmc_cases.xlsb"

#Read the file
data = pd.read_excel(data_dir+case_sheet,engine='pyxlsb',header=None)

In [18]:
# Some sheets have a blank row at top and some don't. By reading without header, dropping the blank rows and assigning the top row as header back, we can fix this. 
# If there is a better solution to this, or later if this problem is resolved if we shift to S3 containers, feel free to edit. 

data = data.dropna(how='all')
data.columns = data.iloc[0]
data = data[1:]

In [21]:
# Count all the non-na (non-empty) values in each columns
nvals = data.notna().sum()
print(nvals)

0
Date                                           100487
S No                                            52926
Ward                                           100487
Test ID (ICMR)                                 100473
Patient name                                        0
Age                                            100487
Gender                                         100487
Mobile Number                                       0
Present Address                                     0
Transferred from Ward                            9015
HealthPost Allocation                           74370
Traceable?                                      98535
Out of Mumbai (Yes / No)                        72999
Assign to other ward                             5828
Person / Official from other ward                   0
Duplicate Entry\n(Yes / No)                     66879
Previous Test ID (only if Duplicate is Yes)      3262
Patient Type (Index / Contact)                  70012
Patient Status            